In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from IPython.display import display,HTML
import os
from ipywidgets import IntProgress


# steps:

1. read files into pandas dataframe, and store that dataframe in a pickle file

2. split them into train/dev/test sets, divided by time, make sure they are all from different distributions.
   train: x < 2017-03-01
   dev: 2017-03-01 <= x < 2017-06-01
   test: 2017-06-01 <= x < 2017-08-01

3. feed training set into a RNN, to see if it can generalize some patterns which can be used to predict on dev set.
   if the model can correctly predict dev labels without training in dev set, it means it worked.

4. test on test set, and after that maybe test on real data.

In [2]:
sh_datapath = '/home/liusida/ipython/fund_scanner/data/a_stock/XSHG/'
sz_datapath = '/home/liusida/ipython/fund_scanner/data/a_stock/XSHE/'
pickle_path = '/home/liusida/ipython/fund_scanner/data/a_stock/using_rnn_to_predict.pickle'

date_mark = ['1990-01-01', '2017-03-01', '2017-06-01', '2017-08-01']

In [5]:
def filelist(datapath):
    df_filelist = pd.DataFrame(os.listdir( datapath ))
    return df_filelist[df_filelist[0].str.endswith('.csv')][0].tolist()
pd.set_option('display.max_columns', 0)
pd.set_option('display.max_rows', 10)

for file in filelist( sz_datapath )[:4]:
    filepath = os.path.join(sz_datapath, file)
    df = pd.read_csv(filepath, encoding='GBK')
    stock_name = df.loc[0, '股票名称']
    df = df.loc[:, ['交易日期', '开盘价','最高价','最低价','收盘价','后复权价','成交额','流通市值','总市值']]
    df['交易日期'] = pd.to_datetime(df['交易日期'])
    df_train = df.loc[df['交易日期']<pd.to_datetime(date_mark[1])]
    df_dev = df.loc[((df['交易日期']>=pd.to_datetime(date_mark[1])) & (df['交易日期']<pd.to_datetime(date_mark[2]))) ]
    df_test = df.loc[df['交易日期']>pd.to_datetime(date_mark[2])]
    display(stock_name)
    display(df_dev)


'立昂技术'

,交易日期,开盘价,最高价,最低价,收盘价,后复权价,成交额,流通市值,总市值
22,2017-05-31,32.30,32.87,31.20,31.30,31.30,4.312993e+07,8.044100e+08,3.208250e+09
23,2017-05-26,31.43,31.97,30.84,31.32,31.32,4.614607e+07,8.049240e+08,3.210300e+09
24,2017-05-25,32.48,32.60,30.15,31.65,31.65,6.665282e+07,8.134050e+08,3.244125e+09
25,2017-05-24,32.88,34.12,30.60,32.48,32.48,5.759300e+07,8.347360e+08,3.329200e+09
26,2017-05-23,37.02,37.48,33.31,33.31,33.31,5.350095e+07,8.560670e+08,3.414275e+09
...,...,...,...,...,...,...,...,...,...
78,2017-03-07,46.65,48.36,45.88,47.50,47.50,7.198051e+08,1.220750e+09,4.868750e+09
79,2017-03-06,42.48,48.40,41.45,48.40,48.40,7.539075e+08,1.243880e+09,4.961000e+09
80,2017-03-03,44.11,48.40,40.00,44.00,44.00,7.860246e+08,1.130800e+09,4.510000e+09
81,2017-03-02,40.10,44.11,38.50,44.11,44.11,7.072740e+08,1.133627e+09,4.521275e+09


'大康农业'

,交易日期,开盘价,最高价,最低价,收盘价,后复权价,成交额,流通市值,总市值
0,2017-03-10,3.50,3.51,3.48,3.50,53.189685,32642461.60,5.411308e+09,1.919880e+10
1,2017-03-09,3.53,3.53,3.49,3.50,53.189685,37763642.35,5.411308e+09,1.919880e+10
2,2017-03-08,3.55,3.56,3.52,3.53,53.645597,34506294.21,5.457690e+09,1.936336e+10
3,2017-03-07,3.56,3.56,3.54,3.55,53.949538,30904616.42,5.488612e+09,1.947307e+10
4,2017-03-06,3.53,3.56,3.52,3.56,54.101509,52506547.84,5.504073e+09,1.952793e+10
5,2017-03-03,3.52,3.54,3.51,3.54,53.797567,38777275.59,5.473151e+09,1.941822e+10
6,2017-03-02,3.56,3.56,3.52,3.53,53.645597,51263349.80,5.457690e+09,1.936336e+10
7,2017-03-01,3.57,3.58,3.55,3.56,54.101509,50093800.01,5.504073e+09,1.952793e+10


'雅化集团'

,交易日期,开盘价,最高价,最低价,收盘价,后复权价,成交额,流通市值,总市值
43,2017-05-31,9.35,9.37,8.70,8.71,56.909910,3.672300e+08,7.008544e+09,8.361600e+09
44,2017-05-26,9.49,9.68,9.16,9.16,59.850146,3.328754e+08,7.370639e+09,8.793600e+09
45,2017-05-25,9.56,9.68,9.17,9.49,62.006320,3.738101e+08,7.636175e+09,9.110400e+09
46,2017-05-24,9.20,9.74,9.01,9.62,62.855721,3.823489e+08,7.740780e+09,9.235200e+09
47,2017-05-23,9.56,9.99,9.47,9.51,62.136997,4.740151e+08,7.652268e+09,9.129600e+09
...,...,...,...,...,...,...,...,...,...
99,2017-03-07,6.97,7.02,6.91,6.93,45.279641,5.185856e+07,5.385927e+09,6.652800e+09
100,2017-03-06,7.00,7.08,6.96,7.00,45.737011,6.811835e+07,5.440331e+09,6.720000e+09
101,2017-03-03,6.81,7.08,6.77,7.00,45.737011,1.169317e+08,5.440331e+09,6.720000e+09
102,2017-03-02,6.77,6.92,6.77,6.83,44.626255,7.297549e+07,5.308208e+09,6.556800e+09


'民和股份'

,交易日期,开盘价,最高价,最低价,收盘价,后复权价,成交额,流通市值,总市值
43,2017-05-31,11.20,11.25,10.75,10.85,22.608482,9.230301e+07,2.236750e+09,3.277206e+09
44,2017-05-26,11.10,11.20,10.90,11.10,23.129415,4.511247e+07,2.288289e+09,3.352718e+09
45,2017-05-25,10.95,11.22,10.72,11.14,23.212764,5.808447e+07,2.296535e+09,3.364799e+09
46,2017-05-24,10.83,11.08,10.50,11.05,23.025228,5.204537e+07,2.277981e+09,3.337615e+09
47,2017-05-23,11.25,11.45,10.81,10.92,22.754343,8.251696e+07,2.251181e+09,3.298349e+09
...,...,...,...,...,...,...,...,...,...
99,2017-03-07,19.91,20.92,19.91,20.91,43.570816,3.917005e+08,4.310641e+09,6.315795e+09
100,2017-03-06,19.93,20.09,19.71,19.97,41.612109,1.711776e+08,4.116858e+09,6.031871e+09
101,2017-03-03,19.73,20.08,19.61,19.95,41.570435,2.094495e+08,4.112735e+09,6.025830e+09
102,2017-03-02,20.10,20.10,19.66,19.75,41.153688,2.476635e+08,4.071504e+09,5.965421e+09
